In [1]:
import fitz
import pandas as pd
import os
import unicodedata
from IPython.display import clear_output
import time

In [2]:
short_words=["a",'i',"of","on","in","ok","so","be","to","me",'we','he','ai','am','an','as','at','bi','by','ed','en','hi','id','if','in','is','it','my','or','os','pi','us','no']
short_words = short_words + [word.upper() for word in short_words] + [word.capitalize() for word in short_words]
def has_numbers(token):
    return any(char.isdigit() for char in token)

In [3]:
data = pd.DataFrame(columns=["pages"])

folder = "../dat/books/selected10/"


start_pages = []
end_pages = []
for book in os.listdir(folder):
    doc = fitz.open("../dat/books/selected10/"+book)

    # table of content
    toc = doc.get_toc()
    tocs_1level = [item for item in toc if item[0]==1]
    print(tocs_1level)
    time.sleep(4)

    # input the number of page where the main content starts
    start = int(input())
    start_pages.append(start)

    # input the number of the next page after the main content
    end = int(input())
    end_pages.append(end)

    # read the text from pdf and remove unnecessary content and noise
    text=[]
    for i,page in enumerate(doc):
        if (i+1>=start) and (i+1<end):

            page_text = page.get_text()

            page_text = unicodedata.normalize("NFKD", page_text)
            page_text = page_text.encode("ascii", "ignore")
            page_text = page_text.decode()
            page_text = page_text.replace("-\n","")
            page_text = page_text.replace("\n"," ")

            text.append(page_text)
        else:
            text.append("")
    

    cleaned_text=[]
    for page in text:
        cleaned_tokens = []
        for token in page.split(" "):
            if not ((has_numbers(token)) or ((len(token) <=2) and (token not in short_words))):
                cleaned_tokens.append(token)
        cleaned_text.append(" ".join(cleaned_tokens))
    
    data.loc[book[:-4],"pages"]=cleaned_text
    clear_output(wait=True)

[[1, 'Cover  ', 1], [1, 'Title\r', 2], [1, 'Preface\r', 4], [1, 'Contents\r', 7], [1, '1. Linear Equations\r', 10], [1, '2. Vector Spaces\r', 37], [1, '3. Linear Transformations\r', 76], [1, '4. Polynomials\r', 126], [1, '5. Determinants\r', 149], [1, '6. Elementary Canonical Forms\r', 190], [1, '7. The Rational and Jordan Forms\r', 236], [1, '8. Inner Product Spaces\r', 279], [1, '9. Operators on Inner Product Spaces\r', 328], [1, '10. Bilinear Forms\r', 368], [1, 'Appendix\r', 395], [1, 'Bibliography\r', 409], [1, 'Index\r', 410]]


In [4]:
data

,pages
Beezer_First_Course,"[, , , , , , , , , , , , , , Chapter SLE Syste..."
CollegeAlgCoreq-WEB,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
Hefferon_LinAlgebra,"[, , , , , , , , , , Chapter One Linear System..."
interactive_textbook,"[, , , , , , , , , , , , , , , , , , Chapter S..."
Kuttler-LinearAlgebra-AFirstCourse-2017A,"[, , , , , , , , , , , , , , Systems of Equati..."
Linear algebra done right — Axler,"[, , , , , , , , , , , , , , , , , Rene Descar..."
"linear-Cherey, Denton","[, , , , , , , , What is Linear Algebra? Many ..."
Math1410_print,"[, , , , , , , , One thing that student evalua..."
Nicholson-OpenLAWA-2019A,"[, , , , , , , , , , , , , , , , , , , , , , S..."
textbook_Hoffman_Kunze,"[, , , , , , , , , Linear Equations Fields We ..."


In [5]:
data.loc['Beezer_First_Course','pages'][100]

'Beezer: A First Course in Linear Algebra matrix for the original system in Archetype In particular, this demonstrates that this coefficient matrix is singular. There is a lot going on in the last two examples. Come back to them in a while and make some connections with the intervening material. For now, we will summarize and explain some of this behavior with a theorem. Theorem SLSLC Solutions to Linear Systems are Linear Combinations Denote the columns of the matrix A as the vectors An. Then is a solution to the linear system of equations LS(A, if and only if equals the linear combination of the columns of A formed with the entries of [x]n An Proof. The proof of this theorem is as much about a change in notation as it is about making logical deductions. Write the system of equations LS(A, as ... amnxn Notice then that the entry of the coefficient matrix A in row i and column has two names: aij as the coefficient of in equation i of the system and [Aj]i as the i-th entry of the column

In [6]:
start_pages

[15, 109, 11, 19, 15, 18, 9, 9, 23, 10]

In [7]:
end_pages

[595, 1162, 497, 431, 549, 347, 315, 391, 619, 395]

In [8]:
data.to_json("../dat/parsed_books/parsed_books.json")